In [1]:
import os
import json
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei'] # Or any other Chinese characters

In [2]:
name_map_table = {
    "Botnet-ARES.csv": "Botnet-ARES",
    "BruteForce-FTP.csv": "BruteForce-FTP",
    "BruteForce-SSH.csv": "BruteForce-SSH",
    "DoS-GoldenEye.csv": "DoS-GoldenEye",
    "DoS-Hulk.csv": "DoS-Hulk",
    "DoS-Slowhttptest.csv": "DoS-Slowhttptest",
    "DoS-Slowloris.csv": "DoS-Slowloris",
    "Heartbleed-Port444.csv": "Heartbleed-Port444",
    "Infiltration-CoolDisk.csv": "Infiltration-CoolDisk",
    "Infiltration-Dropbox.csv": "Infiltration-Dropbox",
    "PortScan.csv": "PortScan",
    "Web-BruteForce.csv": "Web-BruteForce",
    "Web-SqlInjection.csv": "Web-SqlInjection",
    "Web-XSS.csv": "Web-XSS",
}

name_map_figure = {
    "Botnet-ARES.csv": "ARES",
    "BruteForce-FTP.csv": "FTP",
    "BruteForce-SSH.csv": "SSH",
    "DoS-GoldenEye.csv": "GoldenEye",
    "DoS-Hulk.csv": "Hulk",
    "DoS-Slowhttptest.csv": "Slowhttptest",
    "DoS-Slowloris.csv": "Slowloris",
    "Heartbleed-Port444.csv": "Heartbleed",
    "Infiltration-CoolDisk.csv": "CoolDisk",
    "Infiltration-Dropbox.csv": "Dropbox",
    "PortScan.csv": "PortScan",
    "Web-BruteForce.csv": "BruteForce",
    "Web-SqlInjection.csv": "SqlInjection",
    "Web-XSS.csv": "XSS",
}

method_map = {
    "kmeans": "Whisper",
    "aec": "TAE",
    "aew": "FAE"
}


In [7]:
with open("result/ensemble-cic/benign.csv/1.2-1.2-1.2/all-metrics.json", "r") as f:
    metrics_dict = json.load(f)

all_metrics = []
for attack_type, val1 in metrics_dict.items():
    metric_list = [name_map_table[attack_type.split("/")[-1]]]
    for method, val2 in val1.items():
        metric_list += val2[:4]
    all_metrics.append(metric_list)

column_names = []
for method in ["kmeans", "aec", "aew", "majority", "positive", "weighted"]:
    column_names += [f"{method}_accuracy", f"{method}_precision", f"{method}_recall", f"{method}_f1"]
df = pd.DataFrame(all_metrics, columns=["attack_type"]+column_names)

df.to_csv("result/ensemble-cic/benign.csv/1.2-1.2-1.2/all-metrics.csv", index=False)

## draw line

In [8]:
with open("result/ensemble-cic/benign.csv/all-metrics.json", "r") as f:
    metrics_dict = json.load(f)

metrics = ["准确率", "精确率", "召回率", "F1"]
methods_to_plot = ["kmeans", "aec", "aew"]
save_dir = "result/ensemble-cic/4"
os.makedirs(save_dir, exist_ok=True)
for idx in range(len(metrics)):
    cur_metric = metrics[idx]
    method_values = {method_map[method]: [] for method in methods_to_plot}
    x_lables = []
    for attack_type, val1 in metrics_dict.items():
        x_lables.append(name_map_figure[attack_type.split("/")[-1]])
        for method, val2 in val1.items():
            if method in methods_to_plot:
                method_values[method_map[method]].append(val2[idx])

    # draw figure with matplotlib
    fig = plt.figure()
    plt.title(f"{cur_metric}")
    plt.xlabel("攻击类型")
    plt.ylabel("数值")
    plt.tick_params(axis='x', labelrotation=45)
    for method in methods_to_plot:
        plt.plot(x_lables, method_values[method_map[method]], label=method_map[method],
                 marker=".")
    plt.legend()
    plt.savefig(os.path.join(save_dir, f"{cur_metric}.pdf"))
    plt.close(fig)

## draw radar

In [7]:
scale_str = f"{1.5}-{1.5}-{1.5}"

with open(f"result/ensemble-cic/benign.csv/{scale_str}/all-metrics.json", "r") as f:
    metrics_dict = json.load(f)

metrics = ["准确率", "精确率", "召回率", "F1"]
methods_to_plot = ["kmeans", "aec", "aew"]
save_dir = f"result/ensemble-cic/{scale_str}"
os.makedirs(save_dir, exist_ok=True)
for idx in range(len(metrics)):
    cur_metric = metrics[idx]
    method_values = {method_map[method]: [] for method in methods_to_plot}
    x_lables = []
    for attack_type, val1 in metrics_dict.items():
        x_lables.append(name_map_figure[attack_type.split("/")[-1]])
        for method, val2 in val1.items():
            if method in methods_to_plot:
                method_values[method_map[method]].append(val2[idx])
    
    # draw radar figure with matplotlib
    fig = plt.figure()
    num_attack = len(x_lables)
    raw_angles = np.linspace(0, 2*np.pi, num_attack, endpoint=False)
    angles = np.concatenate((raw_angles, [raw_angles[0]]))
    ax = fig.add_subplot(111, polar=True)
    for method in methods_to_plot:
        values = method_values[method_map[method]]
        values = np.concatenate((values, [values[0]]))
        ax.plot(angles, values, '.-', linewidth=1, label=method_map[method])
        ax.fill(angles, values, alpha=0.25)
    ax.set_thetagrids(raw_angles * 180/np.pi, x_lables)
    ax.set_title(f"{cur_metric}")
    ax.grid(True)
    plt.legend()
    plt.savefig(os.path.join(save_dir, f"{cur_metric}-radar.pdf"))
    plt.close(fig)